# Regression clustering and change point detection

In [ ]:
# import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from ipywidgets import *
import numpy as np
from scipy import stats


In [ ]:
gm_csv = pd.read_csv('./tmp/160-IW1-414-s1-asc1-v2020.csv')
# remove whitespaces from column names 
gm_csv.rename(columns=lambda x: x.strip(), inplace=True)
#transposer - rowes are columns and vice versa
gm_csv_t = gm_csv.T
# set the 'pid' as the column name
gm_csv_t.rename(columns=gm_csv_t.iloc[0], inplace=True)
#remove all rows which are not dates - first date is on entry #21
gm_csv_t_dates= gm_csv_t.iloc[21: , :]
gm_csv_t_dates.columns = gm_csv_t_dates.columns.astype(str)

In [ ]:
#need to make a copy of the dataframe so that I don't mess it up
gm_copy = gm_csv_t_dates.copy()

### Linear regression on timeseries - what are the trends that we are trying to pick up??

* Ascending, descending, constant or irregular
* Seasonality trends
* Points of change in the behaviour

http://kdd.ics.uci.edu/databases/synthetic_control/synthetic_control.data.html

We could potentially use this data to pretrain an algorithm?
https://www.andreaperlato.com/tspost/time-series-classification/

In [ ]:
# create a new column with the timestep number to make it easier to work with instead of dates
pd.to_datetime(gm_csv_t_dates.index)
df = gm_csv_t_dates.copy()
df['timeseries_step'] = np.arange(0, len(gm_csv_t_dates))
first_column = df.pop('timeseries_step')
df.insert(0, 'timeseries_step', first_column)

Can we just do a simple linear regression on the data??

In [ ]:
all_y_reg = []
for i in range(1,len(df.columns)):
    X = df.loc[:, ['timeseries_step']]  # features
    y = df.iloc[:, i]  # target
    x = X.values.squeeze()
    x = x.astype(float)
    y = y.values.astype(float)
    res = stats.linregress(x, y)
    all_y_reg.append(res)


In [ ]:
# note that the order follows that of the dataframe columns
all_rvalue = []
all_slope_values = []
all_intercept_values = []
all_pvalue = []
all_stderr = []
all_intercept_stderr = []

for i in range(len(all_y_reg)):
    all_rvalue.append(all_y_reg[i].rvalue)
    all_slope_values.append(all_y_reg[i].slope)
    all_intercept_values.append(all_y_reg[i].intercept)
    all_pvalue.append(all_y_reg[i].pvalue)
    all_stderr.append(all_y_reg[i].stderr)
    all_intercept_stderr.append(all_y_reg[i].intercept_stderr)

    

In [ ]:
# create a pandas dataframe with the r values and the same column names (point IDs) as the original dataframe
df_all_rvalue = pd.DataFrame(all_rvalue)
df_all_rvalue.index = gm_csv_t_dates.columns
df_all_rvalue['slope'] = all_slope_values
df_all_rvalue['intercept'] = all_intercept_values
df_all_rvalue['pvalue'] = all_pvalue
df_all_rvalue['stderr'] = all_stderr
df_all_rvalue['intercept_stderr'] = all_intercept_stderr
df_all_rvalue = df_all_rvalue.rename(columns={0: 'rvalue'})
df_all_rvalue.index.name = 'pid'
df_linreg = df_all_rvalue

### Clustering the different types of regression. 
The parameters are quite arbitraty, mostly relying on the value of the slope
and the r value. 
It would be worth studying the sensitivity of these parameters. 

In [ ]:
linear_ascending = []
linear_descending = []
linear_constant = []
irregular = []

for i in range(len(df_linreg)):
    if df_linreg.rvalue.iloc[i]>0.9:
        linear_ascending.append(df_linreg.index[i])
    elif df_linreg.rvalue.iloc[i]<-0.9:
        linear_descending.append(df_linreg.index[i])
    elif (df_linreg.rvalue.iloc[i]<0.025 and df_linreg.rvalue.iloc[i]>-0.025) and (-0.01<df_linreg.slope.iloc[i]<0.01):
        linear_constant.append(df_linreg.index[i])
    else:
        irregular.append(df_linreg.index[i])

In [ ]:
print(f'ascending: {len(linear_ascending)}, descending: {len(linear_descending)},\
constant: {len(linear_constant)}, irregular:{len(irregular)}')

In [ ]:
### WIDGET 
plt.rcParams['figure.figsize'] = [7,4]

column_name = gm_csv_t_dates.columns

def plot_data(column_number):
    plt.plot(gm_csv_t_dates.iloc[:,column_number].values)
    y_regression = df_all_rvalue.slope[column_number]*x + df_all_rvalue.intercept[column_number]
    plt.plot(y_regression)
    
interact(plot_data, column_number=(0,len(column_name)-1,1))

In [ ]:
plt.plot(gm_csv_t_dates[irregular[0]].values, label='ground motion data')
y_regression = df_all_rvalue.slope.loc[irregular[0]]*x + df_all_rvalue.intercept.loc[irregular[0]]
plt.plot(y_regression, label='regression')
plt.legend()
plt.xlabel('Number of days')
plt.ylabel('Displacement (mm)')
#plt.savefig('./figures/irregular_ground_motion_regression.png')

#### Widgets to show the 4 different classes and the regression line 

In [ ]:
### WIDGET - ASCENDING
plt.rcParams['figure.figsize'] = [7,4]

column_name = gm_csv_t_dates.columns

def plot_data(column_number):
    plt.plot(gm_csv_t_dates[linear_ascending[column_number]].values)
    y_regression = df_all_rvalue.slope.loc[linear_ascending[column_number]]*x + df_all_rvalue.intercept.loc[linear_ascending[column_number]]
    plt.plot(y_regression)
    plt.xlabel('Time (days)')
    plt.ylabel('Ground motion (mm)')
    plt.title('Linear Ascending')
    
interact(plot_data, column_number=(0,len(linear_ascending)-1,1))

In [ ]:
### WIDGET - ASCENDING
plt.rcParams['figure.figsize'] = [7,4]

column_name = gm_csv_t_dates.columns

def plot_data(column_number):
    plt.plot(gm_csv_t_dates[linear_descending[column_number]].values)
    y_regression = df_all_rvalue.slope.loc[linear_descending[column_number]]*x + df_all_rvalue.intercept.loc[linear_descending[column_number]]
    plt.plot(y_regression)
    plt.xlabel('Time (days)')
    plt.ylabel('Ground motion (mm)')
    plt.title('Linear Descending')
    
interact(plot_data, column_number=(0,len(linear_descending)-1,1))

In [ ]:
### WIDGET - ASCENDING
plt.rcParams['figure.figsize'] = [7,4]

column_name = gm_csv_t_dates.columns

def plot_data(column_number):
    plt.plot(gm_csv_t_dates[linear_constant[column_number]].values)
    y_regression = df_all_rvalue.slope.loc[linear_constant[column_number]]*x + df_all_rvalue.intercept.loc[linear_constant[column_number]]
    plt.plot(y_regression)
    plt.xlabel('Time (days)')
    plt.ylabel('Ground motion (mm)')
    plt.title('Linear Constant')
    
interact(plot_data, column_number=(0,len(linear_constant)-1,1))

In [ ]:
### WIDGET - ASCENDING
plt.rcParams['figure.figsize'] = [7,4]

column_name = gm_csv_t_dates.columns

def plot_data(column_number):
    plt.plot(gm_csv_t_dates[irregular[column_number]].values)
    y_regression = df_all_rvalue.slope.loc[irregular[column_number]]*x + df_all_rvalue.intercept.loc[irregular[column_number]]
    plt.plot(y_regression)
    plt.xlabel('Time (days)')
    plt.ylabel('Ground motion (mm)')
    plt.title('Irregular')
    
interact(plot_data, column_number=(0,len(irregular)-1,1))

In [ ]:
reshaped_column = np.array(gm_csv_t_dates[irregular[1645]])
reshaped_column = reshaped_column.astype('int')

#### Change Point detection 

Detect where the behaviour changes in the data (I am only applying it to 
irregular data, which ic the one that in principle should have changes in 
behaviour.)

Since the regression discrimination hasn't been optimised, it is possible that 
some of the irregular timeseriesare closer to an ascending or descending series 
than to one with changes of behaviour.

I will be using the ruptures package. This has a few algorithms for change detection
but I haven't explored them in great detail. The window method seems to work well for our data. 

In [ ]:
import ruptures as rpt
# RUPTURES PACKAGE
# Changepoint detection with the Pelt search method

points = reshaped_column

model="rbf"
algo = rpt.Pelt(model=model).fit(points)
result = algo.predict(pen=10)
rpt.display(points, result, figsize=(10, 6))
plt.title('Change Point Detection: Pelt Search Method')
plt.xlabel('Number of days')
plt.ylabel('Displacement (mm)')
#plt.show()  
plt.savefig('./figures/change_point_pelt.png')
plt.clf()

#Changepoint detection with the Binary Segmentation search method
model = "l2"  
algo = rpt.Binseg(model=model).fit(points)
my_bkps = algo.predict(n_bkps=10)
# show results
rpt.show.display(points, my_bkps, figsize=(10, 6))
plt.title('Change Point Detection: Binary Segmentation Search Method')
plt.xlabel('Number of days')
plt.ylabel('Displacement (mm)')
#plt.show()
plt.savefig('./figures/change_point_binary_segmentation.png')
plt.clf()
    
    
#Changepoint detection with window-based search method
model = "l2"  
algo = rpt.Window(width=50, model=model).fit(points)
my_bkps = algo.predict(n_bkps=10)
rpt.show.display(points, my_bkps, figsize=(10, 6))
plt.title('Change Point Detection: Window-Based Search Method')
plt.xlabel('Number of days')
plt.ylabel('Displacement (mm)')
#plt.show()
plt.savefig('./figures/change_point_window.png')
plt.clf()


#Changepoint detection with dynamic programming search method
model = "l1"  
algo = rpt.Dynp(model=model, min_size=3, jump=5).fit(points)
my_bkps = algo.predict(n_bkps=10)
rpt.show.display(points, my_bkps, figsize=(10, 6))
plt.title('Change Point Detection: Dynamic Programming Search Method')
plt.xlabel('Number of days')
plt.ylabel('Displacement (mm)')
#plt.show()
plt.savefig('./figures/change_point_dynamic_programming.png')
plt.clf()


## Fourier Analysis
We want to find seasonlity - if any - in the data. 
This will require morein depth analysis - we are going to assume that the time series is regularly space - which it isn't - but just for the sake of a quick analysis. 

We want to get the frequency distribution of the timeseries. 


From here: https://towardsdatascience.com/fourier-transform-for-time-series-292eb887b101

In [ ]:
#y = gm_csv_t_dates[linear_constant[376]]
y = gm_csv_t_dates.iloc[:,451]
x = np.arange(0,len(gm_csv_t_dates))

In [ ]:
plt.plot(x,y)
plt.xlabel('Number of days')
plt.ylabel('Displacement (mm)')
plt.savefig('./figures/time_series_for_fourier.png')


In [ ]:
# apply fast fourier transform and take absolute values

f=abs(np.fft.fft(y))

# get the list of frequencies
num=np.size(x)
freq = [i / num for i in list(range(num))]

# get the list of spectrums
spectrum=f.real*f.real+f.imag*f.imag
nspectrum=spectrum/spectrum[0]

# plot nspectrum per frequency, with a semilog scale on nspectrum
plt.semilogy(freq,nspectrum)
plt.xlabel('Frequency (1/days)')
plt.savefig('./figures/fourier_frequencies.png')

#plt.ylabel('Frequency strength')

In [ ]:
# improve the plot by adding periods in number of days (assuming regular intervals) rather than  frequency
results = pd.DataFrame({'freq': freq, 'nspectrum': nspectrum})
results['period'] = results['freq'] / (1/len(gm_csv_t_dates))
plt.semilogy(results['period'], results['nspectrum'])

### Plotting the data at the actual time intervals

Here it is easier to see where the gaps in the data are and also the spread within the same measument season. 

In [ ]:
import datetime
dates = pd.to_datetime(gm_csv_t_dates.index)

In [ ]:
all_delta = []
for i in range(len(dates)-1):
    d0 = dates[i]
    d1 = dates[i+1]
    delta = d1 - d0
    all_delta.append(delta)


In [ ]:
# fig = plt.gcf()
# fig.set_size_inches(18.5, 10.5)
# plt.plot(gm_csv_t_dates.iloc[:20,534],'.-')


import matplotlib.pyplot as plt
import numpy as np
import datetime
from matplotlib import dates as mdates

# Using Data from OP: tp_pass and azip_pass

# Creating your plot

fig, ax=plt.subplots(1, 1, figsize=(15, 7))
plt.ylabel('Displacement (mm)')
#ax.set_ylim(-185, 185)
ax.scatter(dates, gm_csv_t_dates.iloc[:,534], color="b", alpha=1.0, ec="k")

# Minor ticks every month.
fmt_month = mdates.MonthLocator()
# Minor ticks every year.
fmt_year = mdates.YearLocator()

ax.xaxis.set_minor_locator(fmt_month)
# '%b' to get the names of the month
ax.xaxis.set_minor_formatter(mdates.DateFormatter('%m'))
ax.xaxis.set_major_locator(fmt_year)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m'))

# fontsize for month labels
ax.tick_params(labelsize=8, which='both')
# create a second x-axis beneath the first x-axis to show the year in YYYY format
sec_xaxis = ax.secondary_xaxis(-0.1)
sec_xaxis.xaxis.set_major_locator(fmt_year)
sec_xaxis.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Hide the second x-axis spines and ticks
sec_xaxis.spines['bottom'].set_visible(False)
sec_xaxis.tick_params(length=0, labelsize=10)


#plt.yticks([-180, -120, -60, 0, 60, 120, 180], ["${}^\circ$".format(x) for x in [-180, -120, -60, 0, 60, 120, 180]], fontsize=35)
plt.tight_layout()
#plt.show()
plt.savefig('./figures/timeseries_points_real_timeline_axis.png')